# Data Selection in Series
## Series as dictionary

In [25]:
import pandas as pd
data = pd.Series([1,99,2,3,4,6],index=['b','c','a','l','l','a'])
data

b     1
c    99
a     2
l     3
l     4
a     6
dtype: int64

- notice that indices can repeat, unexpected!

In [26]:
data[['a','b']]

a    2
a    6
b    1
dtype: int64

- and that fancy indexing returns both matches to 'a' 

We can also use dictionary-like Python expressions and methods to examine the keys/indices and values:

In [27]:
'l' in data, 'q' in data

(True, False)

In [28]:
data.keys()

Index(['b', 'c', 'a', 'l', 'l', 'a'], dtype='object')

In [29]:
data.items(),'  ',list(data.items())

(<zip at 0x7f2d04708a00>,
 '  ',
 [('b', 1), ('c', 99), ('a', 2), ('l', 3), ('l', 4), ('a', 6)])

Series objects can even be modified with a dictionary-like syntax. Just as you can extend a dictionary by assigning to a new key, you can extend a Series by assigning to a new index value:

In [30]:
data['p'] = 2

In [31]:
data

b     1
c    99
a     2
l     3
l     4
a     6
p     2
dtype: int64

## Series as one-dimensional array

In [35]:
# slicing by explicit index

# data['l':'a'] 
# ==> ERROR "Cannot get right slice bound for non-unique label: 'a'"

data['c':'p']
# note that the element with index 'p' is included

c    99
a     2
l     3
l     4
a     6
p     2
dtype: int64

In [36]:
# slicing by implicit integer index
data[1:3]
# notice: upper bound is not 

c    99
a     2
dtype: int64

In [37]:
# masking
data[data>5]

c    99
a     6
dtype: int64

In [39]:
# fancy indexing
data[['p','l']]

p    2
l    3
l    4
dtype: int64

mong these, slicing may be the source of the most confusion. Notice that when slicing with an explicit index (i.e., data['a':'c']), the final index is included in the slice, while when slicing with an implicit index (i.e., data[0:2]), the final index is excluded from the slice.